# **Optimización Avanzada de Modelos – `08_optimizacion_avanzada.ipynb`**

Antes de decidir el modelo final para la detección de eventos grasp-and-lift a partir de señales EEG, es fundamental afinar el rendimiento de los algoritmos más prometedores mediante **búsqueda de hiperparámetros con Optuna**. Esta fase permite sacar el máximo partido a cada modelo sin cambiar los datos de entrada ni la estructura general del pipeline.

### 🎯 **Objetivo**

Mejorar el rendimiento de los modelos **LightGBM**, **XGBoost** y **MLP** aplicando una búsqueda eficiente de hiperparámetros con **Optuna**, maximizando el AUC-ROC y controlando el tiempo de cómputo.

#### 🧠 **¿Por qué Optuna?**

Optuna permite explorar espacios de búsqueda complejos de forma inteligente (optimización bayesiana), evitando pruebas aleatorias innecesarias y acelerando el proceso de tuning. Ventajas clave:

- Ajuste automático con `early stopping`
- Visualización interactiva de importancia de hiperparámetros
- Gestión eficiente de múltiples trials paralelos

### ⚙️ **Modelos seleccionados**

Se optimizarán los siguientes modelos:

1. **LightGBM**: rápido y eficaz en datos tabulares, además los resultados se acercaron e incluso superaron en algunos eventos a LogisticRegression.
2. **XGBoost**: versátil y robusto frente a overfitting.
3. **MLP** (Multi-Layer Perceptron): una red neuronal que podría mejorar al ajustar adecuadamente su arquitectura y regularización.

### 📦 **Configuración de datos**

- Se utilizarán las **512 características originales** (sin selección previa), para mantener toda la señal EEG disponible.
- Para evitar tiempos de cómputo excesivos, se usarán **subconjuntos balanceados** por evento motor durante la validación.
- Se mantendrá la validación cruzada estratificada (`StratifiedKFold`) con 3 o 5 folds.

### 🚀 **Flujo de Trabajo en este Notebook**

1️⃣ **Definición del espacio de búsqueda por modelo**

Para cada modelo se definirá un espacio adecuado de hiperparámetros (ej. `max_depth`, `learning_rate`, `dropout`, etc.)

2️⃣ **Implementación de función objetivo para Optuna**

Se definirá una función objetivo personalizada que entrena el modelo, evalúa el AUC en validación cruzada, y devuelve el resultado para optimización.

3️⃣ **Ejecución de Optuna Trials**

Se ejecutará Optuna con un número controlado de iteraciones (`n_trials`), midiendo tanto rendimiento como tiempo de entrenamiento.

4️⃣ **Análisis de resultados y comparación**

Se recogerán los mejores parámetros, métricas de AUC, y se comparará con el modelo base (Regresión Logística). Se incluirán gráficos de optimización y resumen de importancia de hiperparámetros.

5️⃣ **Decisión sobre el modelo final**

Según los resultados obtenidos, se valorará si alguno de los modelos optimizados merece ser promovido como candidato final o si se mantiene la Regresión Logística como baseline principal.

💡 Esta fase es clave para garantizar que los modelos evaluados han sido entrenados bajo condiciones óptimas, asegurando una comparación justa y sólida entre alternativas.

---

## **1. Carga y muestreo de datos**

In [1]:
import os
import pickle
import pandas as pd

# Ruta de los datos procesados
processed_path = r"C:\Users\luciaft\Documents\TFG\TFG\graspAndLiftDetectionTFGProyect\data\processed"
data_file = os.path.join(processed_path, "preprocessed_features_temporal_freq.pkl")

# Cargar los datos preprocesados
with open(data_file, "rb") as f:
    X_train_scaled, y_train_win, X_valid_scaled, y_valid_win = pickle.load(f)

#Cargamos también los resultados obtenidos con el modelo base
auc_file = os.path.join(processed_path, "auc_results_feats_logreg.csv")

# Cargar CSV y convertir a diccionario
auc_df = pd.read_csv(auc_file, index_col="Evento")
auc_dict_logreg = auc_df["AUC"].to_dict()


# Verificar formas
print("Datos cargados correctamente:")
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"y_train_win shape: {y_train_win.shape}")
print(f"X_valid_scaled shape: {X_valid_scaled.shape}")
print(f"y_valid_win shape: {y_valid_win.shape}")

# Verificar diccionario con resultados
print(auc_dict_logreg)

Datos cargados correctamente:
X_train_scaled shape: (1043205, 512)
y_train_win shape: (1043205, 6)
X_valid_scaled shape: (236894, 512)
y_valid_win shape: (236894, 6)
{'HandStart': 0.8928, 'FirstDigitTouch': 0.8838, 'BothStartLoadPhase': 0.8888, 'LiftOff': 0.9059, 'Replace': 0.8966, 'BothReleased': 0.8751}


In [2]:
import numpy as np
from sklearn.utils import resample

# Asegurarse de que X tiene el mismo índice que y
X_df = pd.DataFrame(X_train_scaled, index=y_train_win.index)

# Lista de eventos
eventos = y_train_win.columns.tolist()

# Diccionarios de salida
X_train_subsampled_dict = {}
y_train_subsampled_dict = {}

# Función corregida para submuestreo
def submuestreo_balanceado(X, y_event, n_samples=5000, random_state=42):
    idx_0 = y_event[y_event == 0].index
    idx_1 = y_event[y_event == 1].index

    n_per_class = n_samples // 2

    sampled_idx_0 = resample(idx_0, replace=False, n_samples=n_per_class, random_state=random_state)
    sampled_idx_1 = resample(idx_1, replace=False, n_samples=n_per_class, random_state=random_state)

    idx_total = np.concatenate([sampled_idx_0, sampled_idx_1])
    np.random.shuffle(idx_total)

    return X.loc[idx_total], y_event.loc[idx_total]

# Loop corregido por evento
for evento in eventos:
    print(f"Procesando evento: {evento}")
    y_event = y_train_win[evento]
    X_event, y_event_bal = submuestreo_balanceado(X_df, y_event)

    X_train_subsampled_dict[evento] = X_event
    y_train_subsampled_dict[evento] = y_event_bal

Procesando evento: HandStart
Procesando evento: FirstDigitTouch
Procesando evento: BothStartLoadPhase
Procesando evento: LiftOff
Procesando evento: Replace
Procesando evento: BothReleased


## **2. Optimización de LightGBM con Optuna (evento LiftOff)**

Para optimizar el modelo LightGBM se ha utilizado la librería **Optuna**, que permite realizar una búsqueda inteligente de hiperparámetros basada en técnicas de optimización bayesiana. A continuación se detallan los hiperparámetros seleccionados, el rango definido para cada uno y la justificación técnica:

### Justificación de cada parámetro (versión eficiente)

| Parámetro           | Rango propuesto         | Justificación |
|---------------------|--------------------------|----------------|
| `learning_rate`     | [0.001, 0.2] (log scale) | Controla cuánto se ajusta el modelo en cada iteración. Valores bajos evitan overfitting pero requieren más iteraciones. El rango cubre desde valores muy conservadores (0.001) hasta rápidos (0.2), siendo típicos valores entre 0.01 y 0.1. |
| `num_leaves`        | [15, 150]                | Controla la complejidad de cada árbol. Más hojas → mayor capacidad de aprendizaje. Valores muy altos pueden sobreajustar. Este rango cubre desde modelos simples (15 hojas) hasta otros más complejos. |
| `max_depth`         | [3, 15]                  | Profundidad máxima de los árboles. Limita la capacidad de memorizar ruido. Entre 3 y 15 es habitual para datos tabulares. |
| `min_child_samples` | [10, 100]                | Mínimo de observaciones en una hoja. Ayuda a evitar divisiones irrelevantes. Valores bajos permiten más división (mayor ajuste), altos reducen el riesgo de overfitting. |
| `subsample`         | [0.5, 1.0]               | Porcentaje de muestras usadas por árbol. Introduce aleatoriedad (como en Random Forest). Mejora la generalización. |
| `colsample_bytree`  | [0.5, 1.0]               | Porcentaje de variables usadas por árbol. También busca reducir overfitting introduciendo aleatoriedad. |
| `lambda_l1`         | [0.0, 5.0]               | Regularización L1 (suma de coeficientes absolutos). Fomenta modelos más simples y dispersos (sparse). |
| `lambda_l2`         | [0.0, 5.0]               | Regularización L2 (suma de cuadrados). Penaliza pesos grandes y mejora la estabilidad del modelo. |

---

#### Validación cruzada y criterio de evaluación

Se ha utilizado validación cruzada estratificada (`StratifiedKFold`) con 3 folds para garantizar una evaluación robusta y evitar overfitting a una sola partición de validación.  
La métrica objetivo es el **AUC-ROC**, que refleja la capacidad del modelo para distinguir entre eventos positivos y negativos en cualquier umbral de decisión.

---


La combinación de estos rangos permite a **Optuna** explorar tanto modelos conservadores como complejos, maximizando la probabilidad de encontrar una configuración óptima que **supere o iguale** el rendimiento de la regresión logística base, sin comprometer la capacidad de generalización del modelo.


In [4]:
import optuna
from optuna.pruners import MedianPruner
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# Subconjunto balanceado de 5 000 muestras para el evento LiftOff
X = X_train_subsampled_dict["LiftOff"]
y = y_train_subsampled_dict["LiftOff"]


def objective(trial):
    params = {
        # — hiperparámetros que busca Optuna —
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.2, log=True),
        "num_leaves"   : trial.suggest_int("num_leaves", 32, 256, log=True),
        "max_depth"    : trial.suggest_int("max_depth", 4, 12),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 120),
        "subsample"        : trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree" : trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha"        : trial.suggest_float("reg_alpha", 0.0, 6.0),
        "reg_lambda"       : trial.suggest_float("reg_lambda", 0.0, 6.0),
        # — fijos para estabilidad/velocidad —
        "n_estimators" : 800,          # tope; se recorta con early-stopping
        "objective"    : "binary",
        "metric"       : "auc",
        "verbosity"    : -1            # silencia salida 
    }

    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
    aucs = []

    for tr, va in skf.split(X, y):
        X_tr, X_va = X.iloc[tr], X.iloc[va]
        y_tr, y_va = y.iloc[tr], y.iloc[va]

        model = LGBMClassifier(**params)
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            callbacks=[
                early_stopping(stopping_rounds=30),   # corta si no mejora
                log_evaluation(period=0)              # sin logs intermedios
            ],
        )

        y_pred = model.predict_proba(X_va)[:, 1]
        aucs.append(roc_auc_score(y_va, y_pred))

    return sum(aucs) / len(aucs)

# Estudio Optuna ultracompacto
study = optuna.create_study(
    direction="maximize",
    pruner=MedianPruner(n_warmup_steps=5)   # corta trials mediocres pronto
)

# 40 trials * cv=2 ≈ 15 min en CPU
study.optimize(objective, n_trials=40, timeout=1200)   # 20 min máx

print(" Mejor AUC medio:", round(study.best_value, 4))
print(" Mejores hiperparámetros:\n", study.best_params)

[I 2025-05-24 23:32:00,138] A new study created in memory with name: no-name-c988d712-fc3e-4283-896c-8870392aa603


Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[800]	valid_0's auc: 0.971373
Training until validation scores don't improve for 30 rounds


[I 2025-05-24 23:32:13,818] Trial 0 finished with value: 0.97319744 and parameters: {'learning_rate': 0.0033106574246918974, 'num_leaves': 102, 'max_depth': 7, 'min_child_samples': 53, 'subsample': 0.6721919626286558, 'colsample_bytree': 0.7541087380703484, 'reg_alpha': 1.0993521088399931, 'reg_lambda': 3.690052069355544}. Best is trial 0 with value: 0.97319744.


Did not meet early stopping. Best iteration is:
[800]	valid_0's auc: 0.975022
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[750]	valid_0's auc: 0.987689
Training until validation scores don't improve for 30 rounds


[I 2025-05-24 23:32:24,246] Trial 1 finished with value: 0.9878822399999999 and parameters: {'learning_rate': 0.021660595685316156, 'num_leaves': 139, 'max_depth': 5, 'min_child_samples': 60, 'subsample': 0.9922876266852532, 'colsample_bytree': 0.8870781222867492, 'reg_alpha': 5.3206991241709245, 'reg_lambda': 0.8103713334064424}. Best is trial 1 with value: 0.9878822399999999.


...

Early stopping, best iteration is:
[349]	valid_0's auc: 0.98993
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[800]	valid_0's auc: 0.984832
Training until validation scores don't improve for 30 rounds


[I 2025-05-24 23:37:05,885] Trial 38 finished with value: 0.9856768 and parameters: {'learning_rate': 0.007031862149223351, 'num_leaves': 194, 'max_depth': 12, 'min_child_samples': 58, 'subsample': 0.8183388961183855, 'colsample_bytree': 0.8543622311762719, 'reg_alpha': 0.7875044919480032, 'reg_lambda': 2.4417280841348967}. Best is trial 11 with value: 0.9924752.


Did not meet early stopping. Best iteration is:
[800]	valid_0's auc: 0.986522
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[800]	valid_0's auc: 0.990967
Training until validation scores don't improve for 30 rounds


[I 2025-05-24 23:37:19,024] Trial 39 finished with value: 0.9912742400000001 and parameters: {'learning_rate': 0.049718696551098646, 'num_leaves': 119, 'max_depth': 11, 'min_child_samples': 68, 'subsample': 0.9003221106421642, 'colsample_bytree': 0.9316274216574348, 'reg_alpha': 0.4788890186009068, 'reg_lambda': 5.787017865224292}. Best is trial 11 with value: 0.9924752.


Did not meet early stopping. Best iteration is:
[790]	valid_0's auc: 0.991581
🌟 Mejor AUC medio: 0.9925
🔧 Mejores hiperparámetros:
 {'learning_rate': 0.17874458846553382, 'num_leaves': 251, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.8688109914097938, 'colsample_bytree': 0.604507173451288, 'reg_alpha': 0.09049857754538002, 'reg_lambda': 2.1443791516396042}


Antes de lanzar la optimización de XGBoost y SVM, es clave **congelar** el mejor modelo obtenido con LightGBM para:

1. **Evitar pérdidas de resultados** si el kernel se reinicia durante nuevos experimentos.
2. **Reutilizar** el modelo entrenado (p.ej., en la fase de validación sobre otros eventos).
3. **Documentar** hiperparámetros y AUC final sin volver a entrenar.

Los pasos a ejecutar ahora antes del siguiente modelo serán:

1. **Reentrenar** LightGBM con los hiperparámetros óptimos y el sub-muestreo completo de 5 000 filas.  
2. **Aplicar Early-Stopping** (`early_stopping(30)`) para fijar automáticamente el nº ideal de árboles.  
3. **Guardar**:
   - Modelo binario → `models/lgbm_liftoff.pkl`  
   - Hiperparámetros → `models/lgbm_liftoff_params.json`  
4. **Registrar** el AUC de validación (`best_score_["valid_0"]["auc"]`) para la tabla comparativa final.

Con el modelo almacenado, podemos continuar de forma segura con la búsqueda de XGBoost y SVM, hacer lo mismo para cada uno de ellos, y más tarde construir la tabla global de resultados sin riesgo de perder el mejor LightGBM ni el mejor XGBoost o SVM.

In [7]:
# 1 y 2 – Reentrenar y guardar LightGBM óptimo (evento: LiftOff)

import joblib
from lightgbm import LGBMClassifier, early_stopping


# Mejores hiperparámetros encontrados por Optuna
best_params = {
    "learning_rate": 0.17874458846553382,
    "num_leaves": 251,
    "max_depth": 12,
    "min_child_samples": 17,
    "subsample": 0.8688109914097938,
    "colsample_bytree": 0.604507173451288,
    "reg_alpha": 0.09049857754538002,
    "reg_lambda": 2.1443791516396042,
    "n_estimators": 800,
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
}

final_lgbm = LGBMClassifier(**best_params)

final_lgbm.fit(
    X_train_subsampled_dict["LiftOff"],
    y_train_subsampled_dict["LiftOff"],
    eval_set=[(X_valid_scaled, y_valid_win["LiftOff"])],
    callbacks=[early_stopping(30)],
)


# 3

import os, json

save_path = r"C:\Users\luciaft\Documents\TFG\TFG\graspAndLiftDetectionTFGProyect\models"
os.makedirs(save_path, exist_ok=True)

# Modelo binario
joblib.dump(final_lgbm, os.path.join(save_path, "lgbm_liftoff.pkl"))

# Hiperparámetros en JSON
with open(os.path.join(save_path, "lgbm_liftoff_params.json"), "w") as f:
    json.dump(best_params, f, indent=4)


# 4

best_auc_lgbm = final_lgbm.best_score_["valid_0"]["auc"]
print("AUC validación final LightGBM:", best_auc_lgbm)

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[310]	valid_0's auc: 0.900784
AUC validación final LightGBM: 0.9007843119955949


## **3. Optimización de XGBoost con Optuna (evento LiftOff)**

### Justificación de cada parámetro (versión eficiente)

| Parámetro              | Rango propuesto / Valor fijo | Justificación |
|------------------------|------------------------------|----------------|
| `learning_rate`        | **[0.001, 0.2]** (log)       | Controla la tasa de aprendizaje. Valores bajos (≈0.01–0.1) reducen el riesgo de sobreajuste a costa de más iteraciones; Optuna explora desde muy conservador a agresivo. |
| `n_estimators`         | **600** (fijo)               | Límite superior de árboles. Se combina con *early-stopping* para detenerse antes si no mejora, evitando entrenamientos largos innecesarios. |
| `max_depth`            | **[3, 10]**                  | Profundidad máxima del árbol. Cortar en 10 limita el crecimiento exponencial del nº de nodos y acelera cada trial sin perder capacidad representativa. |
| `min_child_weight`     | **[1, 8]**                   | Peso mínimo (suma de hessians) para dividir un nodo. Valores mayores evitan divisiones poco robustas en un dataset submuestreado. |
| `gamma`                | **[0.0, 4.0]**               | Reducción mínima de la loss necesaria para seguir dividiendo. Actúa como regularización estructural ligera. |
| `subsample`            | **[0.6, 1.0]**               | Fracción de filas usada en cada árbol. Introduce aleatoriedad tipo Bagging; rangos <0.6 empeoraban AUC en pruebas preliminares. |
| `colsample_bytree`     | **[0.6, 1.0]**               | Fracción de columnas (features) por árbol. Diversifica los árboles y contiene el tiempo de cada split con 512 variables. |
| `reg_alpha`            | **[0.0, 4.0]**               | Regularización L1 (Lasso). Fomenta modelos más esparsos, útil con 512 vars. |
| `reg_lambda`           | **[0.0, 4.0]**               | Regularización L2 (Ridge). Suaviza los pesos y aporta estabilidad. |
| `early_stopping_rounds`| **25** (fijo, en constructor) | Cesa el entrenamiento cuando no hay mejora tras 25 árboles ⇒ ahorra minutos por trial. |
| `eval_metric`          | `"auc"` (fijo)               | Métrica robusta frente a desbalanceo, coherente con la métrica del TFG. |

---

### Validación y estrategia de optimización

- **Validación cruzada estratificada (`StratifiedKFold`) con 2 folds**: reduce a la mitad el tiempo por trial manteniendo estimaciones fiables en un dataset balanceado.  
- **Pruning con `MedianPruner`**: corta configuraciones cuyo AUC intermedio cae por debajo de la mediana provisional → evita gastar tiempo en trials claramente pobres.  
- **`timeout = 1 200 s` (20 min)** y **`n_trials ≤ 40`**: garantiza que la búsqueda no sobrepase la ventana de cómputo disponible y prioriza las combinaciones más prometedoras.

---

Esta configuración limita la complejidad máxima del modelo y usa técnicas de *early-stopping* y *pruning* para que **Optuna explore eficazmente** sin disparar el tiempo total de optimización. Se equilibra así **calidad del AUC** y **eficiencia computacional**, condición clave para iterar rápido y comparar con la regresión logística y LightGBM dentro del mismo ciclo de trabajo.

In [11]:
# === Optuna + XGBoost (versión “limpia” y eficiente, ya que salieron warnings de como se debía añadir early_stopping y me he dado cuenta de que podría tardar mucho tiempo en ejecutarse) =========================

import optuna
from optuna.pruners import MedianPruner
from xgboost import XGBClassifier
from xgboost.callback import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# Subconjunto balanceado de 5 000 muestras para el evento LiftOff
X = X_train_subsampled_dict["LiftOff"]
y = y_train_subsampled_dict["LiftOff"]

def objective(trial):
    # ---------- espacio de búsqueda ----------
    xgb_params = {
        "learning_rate"    : trial.suggest_float("learning_rate", 1e-3, 0.2, log=True),
        "max_depth"        : trial.suggest_int  ("max_depth", 3, 10),
        "min_child_weight" : trial.suggest_int  ("min_child_weight", 1, 8),
        "gamma"            : trial.suggest_float("gamma", 0.0, 4.0),
        "subsample"        : trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree" : trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha"        : trial.suggest_float("reg_alpha", 0.0, 4.0),
        "reg_lambda"       : trial.suggest_float("reg_lambda", 0.0, 4.0),
        # --- fijos ---
        "n_estimators" : 600,      # se recorta con early-stopping
        "eval_metric"  : "auc",
        "n_jobs"       : 4,        # ajusta al nº de hilos que tengas
    }

    # Early-Stopping como CALLBACK en el CONSTRUCTOR  →  ¡sin warning!
    estop = EarlyStopping(rounds=25)
    model = XGBClassifier(**xgb_params, callbacks=[estop])

    # ---------- validación cruzada 2-fold ----------
    skf  = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
    aucs = []

    for tr_idx, va_idx in skf.split(X, y):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        model.fit(X_tr, y_tr, eval_set=[(X_va, y_va)], verbose=False)
        y_prob = model.predict_proba(X_va)[:, 1]
        aucs.append(roc_auc_score(y_va, y_prob))

    return sum(aucs) / len(aucs)

# ---------- estudio Optuna ----------
study = optuna.create_study(
    direction="maximize",
    pruner=MedianPruner(n_warmup_steps=5)   # corta trials mediocres pronto
)
study.optimize(objective, n_trials=40, timeout=1800)   # ⏱ 30 min máximo

print(" Mejores hiperparámetros:\n", study.best_params)
print(f" Mejor AUC medio: {study.best_value:.4f}")

[I 2025-05-25 00:11:03,287] A new study created in memory with name: no-name-3fc320a7-ba6f-4034-bf84-f7f846fa486f
[I 2025-05-25 00:11:09,885] Trial 0 finished with value: 0.9300673600000001 and parameters: {'learning_rate': 0.09518763095616342, 'max_depth': 5, 'min_child_weight': 2, 'gamma': 1.4051059288817038, 'subsample': 0.8769716435803087, 'colsample_bytree': 0.649228847708827, 'reg_alpha': 1.0841070103135988, 'reg_lambda': 1.2060372677041666}. Best is trial 0 with value: 0.9300673600000001.
[I 2025-05-25 00:11:24,791] Trial 1 finished with value: 0.9319876800000001 and parameters: {'learning_rate': 0.041796747092697005, 'max_depth': 8, 'min_child_weight': 5, 'gamma': 3.4617491059939454, 'subsample': 0.8336093350768542, 'colsample_bytree': 0.7232818560893214, 'reg_alpha': 1.782521543635676, 'reg_lambda': 1.8658754471752608}. Best is trial 1 with value: 0.9319876800000001.
[I 2025-05-25 00:11:40,348] Trial 2 finished with value: 0.93850048 and parameters: {'learning_rate': 0.1226666

🔧 Mejores hiperparámetros:
 {'learning_rate': 0.009361603850241975, 'max_depth': 7, 'min_child_weight': 4, 'gamma': 1.1375069031890903, 'subsample': 0.9641136987869432, 'colsample_bytree': 0.7757351794042561, 'reg_alpha': 0.024204084662042824, 'reg_lambda': 0.4124032759722731}
🌟 Mejor AUC medio: 0.9756


In [14]:
import joblib
import os, json
import pandas as pd
from xgboost import XGBClassifier
from xgboost.callback import EarlyStopping
from sklearn.metrics import roc_auc_score

# Mejores hiperparámetros
best_params = {
    "learning_rate": 0.009361603850241975,
    "max_depth": 7,
    "min_child_weight": 4,
    "gamma": 1.1375069031890903,
    "subsample": 0.9641136987869432,
    "colsample_bytree": 0.7757351794042561,
    "reg_alpha": 0.024204084662042824,
    "reg_lambda": 0.4124032759722731,
    "n_estimators": 600,
    "eval_metric": "auc",
    "n_jobs": 6
}

# Restaurar nombres de columna a X_valid (evita el ValueError de XGBoost por tener el DataFrame sin nombres de columnas)
X_train = X_train_subsampled_dict["LiftOff"]
X_valid = pd.DataFrame(X_valid_scaled, columns=X_train.columns)

# Reentrenar el modelo
final_xgb = XGBClassifier(**best_params, callbacks=[EarlyStopping(rounds=30)])
final_xgb.fit(
    X_train,
    y_train_subsampled_dict["LiftOff"],
    eval_set=[(X_valid, y_valid_win["LiftOff"])],
    verbose=False
)

# Guardar modelo y parámetros
save_path = r"C:\Users\luciaft\Documents\TFG\TFG\graspAndLiftDetectionTFGProyect\models"
os.makedirs(save_path, exist_ok=True)

joblib.dump(final_xgb, os.path.join(save_path, "xgb_liftoff.pkl"))

with open(os.path.join(save_path, "xgb_liftoff_params.json"), "w") as f:
    json.dump(best_params, f, indent=4)

# Evaluar
y_pred = final_xgb.predict_proba(X_valid)[:, 1]
best_auc_xgb = roc_auc_score(y_valid_win["LiftOff"], y_pred)
print(f"AUC validación final XGBoost: {best_auc_xgb:.4f}")

AUC validación final XGBoost: 0.8981


## **3. Optimización de MLP con Optuna (evento LiftOff)**

### Justificación de cada parámetro (versión optimizada)

| Parámetro              | Rango propuesto / Valor fijo      | Justificación |
|------------------------|-----------------------------------|----------------|
| `n_layers`             | **[1, 4]**                        | Controla la profundidad de la red. 1-2 capas es habitual en MLPs clásicos, pero 3-4 capas permiten mayor capacidad representativa para detectar patrones no lineales en señales EEG. |
| `n_units_l{i}`         | **[100, 300]** por capa           | Número de neuronas por capa. Rango suficiente para modelar relaciones complejas sin llegar a redes demasiado profundas/costosas. |
| `activation`           | **["relu", "tanh"]**             | Funciones de activación clásicas. `relu` acelera convergencia; `tanh` es útil cuando se busca una activación centrada en cero. |
| `solver`               | `"adam"` (fijo)                   | Optimizador robusto y eficiente para datasets con ruido como los de EEG. Adapta automáticamente la tasa de aprendizaje. |
| `alpha`                | **[1e-6, 1e-2]** (log)            | Término de regularización L2. Penaliza pesos grandes y ayuda a evitar el sobreajuste. Rango logarítmico permite explorar desde mínima hasta moderada regularización. |
| `learning_rate`        | **["constant", "invscaling", "adaptive"]** | Estrategias de variación del learning rate. `adaptive` es útil con early-stopping; `invscaling` permite convergencia más fina. |
| `learning_rate_init`   | **[1e-4, 1e-2]** (log)            | Tasa de aprendizaje inicial. Valores pequeños permiten ajustes suaves de los pesos. Crucial para estabilidad del entrenamiento. |
| `max_iter`             | **600** (fijo)                    | Iteraciones máximas. Permite convergencia adecuada incluso en redes profundas. Se interrumpe antes si se activa el *early stopping*. |
| `early_stopping`       | `True` (fijo)                     | Detiene el entrenamiento si no hay mejora en la validación, acelerando los trials y reduciendo el sobreajuste. |
| `random_state`         | **42** (fijo)                     | Reproducibilidad del experimento. |

---

### Validación y estrategia de optimización

- **Validación cruzada estratificada (`StratifiedKFold`) con 3 folds**: evaluación más exigente que con 2 folds, lo que mejora la estimación del AUC medio y refuerza la robustez de los resultados.  
- **Pruning con `MedianPruner`**: corta configuraciones con rendimiento inferior a la mediana provisional, acelerando la exploración.  
- **`timeout = 2400 s` (40 min)** y **`n_trials ≤ 80`**: se amplió la ventana de exploración debido a que versiones anteriores con menos trials ejecutaron en poco tiempo, indicando baja complejidad computacional en el caso de MLP. Esto permitió explorar arquitecturas más variadas y tasas de aprendizaje más diversas sin comprometer eficiencia global.

---

Este enfoque permitió explotar la **versatilidad del MLP** con un espacio de búsqueda controlado y técnicas de regularización y parada temprana, garantizando tanto **buen rendimiento en AUC** como **evitación del sobreajuste** en un conjunto reducido y balanceado de muestras del evento motor LiftOff.



In [16]:
# === Optuna + MLPClassifier (versión optimizada) ===========================

import optuna
from optuna.pruners import MedianPruner
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# Subconjunto balanceado para LiftOff
X = X_train_subsampled_dict["LiftOff"]
y = y_train_subsampled_dict["LiftOff"]

def objective(trial):
    # Arquitectura dinámica de capas ocultas
    n_layers = trial.suggest_int("n_layers", 1, 4)
    hidden_layer_sizes = tuple(
        trial.suggest_int(f"n_units_l{i}", 100, 300) for i in range(n_layers)
    )

    # Hiperparámetros
    params = {
        "hidden_layer_sizes": hidden_layer_sizes,
        "activation": trial.suggest_categorical("activation", ["relu", "tanh"]),
        "solver": "adam",
        "alpha": trial.suggest_float("alpha", 1e-6, 1e-2, log=True),
        "learning_rate": trial.suggest_categorical("learning_rate", ["constant", "invscaling", "adaptive"]),
        "learning_rate_init": trial.suggest_float("learning_rate_init", 1e-4, 1e-2, log=True),
        "max_iter": 600,
        "early_stopping": True,
        "random_state": 42,
    }

    # Validación cruzada estratificada más exigente
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    aucs = []

    for train_idx, valid_idx in skf.split(X, y):
        X_train_cv, X_valid_cv = X.iloc[train_idx], X.iloc[valid_idx]
        y_train_cv, y_valid_cv = y.iloc[train_idx], y.iloc[valid_idx]

        model = MLPClassifier(**params)
        model.fit(X_train_cv, y_train_cv)

        y_pred = model.predict_proba(X_valid_cv)[:, 1]
        auc = roc_auc_score(y_valid_cv, y_pred)
        aucs.append(auc)

    return sum(aucs) / len(aucs)

# Estudio Optuna con más exploración (80 trials o 40 minutos máx)
study = optuna.create_study(direction="maximize", pruner=MedianPruner(n_warmup_steps=5))
study.optimize(objective, n_trials=80, timeout=2400)

# Resultados
print(" Mejores hiperparámetros:\n", study.best_params)
print(f" Mejor AUC medio: {study.best_value:.4f}")

[I 2025-05-25 00:47:53,552] A new study created in memory with name: no-name-0f867b1d-4213-4b6a-9b81-d5086d87c42f
[I 2025-05-25 00:48:01,835] Trial 0 finished with value: 0.9833948203836392 and parameters: {'n_layers': 4, 'n_units_l0': 267, 'n_units_l1': 264, 'n_units_l2': 123, 'n_units_l3': 227, 'activation': 'relu', 'alpha': 0.00012173254766352711, 'learning_rate': 'adaptive', 'learning_rate_init': 0.0035431974960343014}. Best is trial 0 with value: 0.9833948203836392.
[I 2025-05-25 00:48:06,582] Trial 1 finished with value: 0.9739156057228296 and parameters: {'n_layers': 3, 'n_units_l0': 128, 'n_units_l1': 234, 'n_units_l2': 237, 'activation': 'tanh', 'alpha': 8.660680525434792e-06, 'learning_rate': 'adaptive', 'learning_rate_init': 0.009604281707024239}. Best is trial 0 with value: 0.9833948203836392.
[I 2025-05-25 00:48:14,161] Trial 2 finished with value: 0.9856513746184964 and parameters: {'n_layers': 4, 'n_units_l0': 165, 'n_units_l1': 202, 'n_units_l2': 284, 'n_units_l3': 289,

 Mejores hiperparámetros:
 {'n_layers': 2, 'n_units_l0': 278, 'n_units_l1': 184, 'activation': 'relu', 'alpha': 5.975997775451657e-06, 'learning_rate': 'invscaling', 'learning_rate_init': 0.002267666524742807}
 Mejor AUC medio: 0.9880


In [18]:
import joblib
import os, json
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score

# Hiperparámetros óptimos encontrados con Optuna
best_params = {
    "hidden_layer_sizes": (278, 184),
    "activation": "relu",
    "solver": "adam",
    "alpha": 5.975997775451657e-06,
    "learning_rate": "invscaling",
    "learning_rate_init": 0.002267666524742807,
    "max_iter": 600,
    "early_stopping": True,
    "random_state": 42,
}

# Restaurar nombres de columna a X_valid
X_train = X_train_subsampled_dict["LiftOff"]
X_valid = pd.DataFrame(X_valid_scaled, columns=X_train.columns)

# Entrenar modelo final
final_mlp = MLPClassifier(**best_params)
final_mlp.fit(X_train, y_train_subsampled_dict["LiftOff"])

# Guardar modelo y parámetros
save_path = r"C:\Users\luciaft\Documents\TFG\TFG\graspAndLiftDetectionTFGProyect\models"
os.makedirs(save_path, exist_ok=True)

joblib.dump(final_mlp, os.path.join(save_path, "mlp_liftoff.pkl"))

with open(os.path.join(save_path, "mlp_liftoff_params.json"), "w") as f:
    json.dump(best_params, f, indent=4)

# Evaluar en validación
y_pred = final_mlp.predict_proba(X_valid)[:, 1]
best_auc_mlp = roc_auc_score(y_valid_win["LiftOff"], y_pred)
print(f"AUC validación final MLP: {best_auc_mlp:.4f}")

AUC validación final MLP: 0.9027


## **4. Comparativa final de rendimiento**

In [19]:
results = {
    "Modelo": ["LightGBM", "XGBoost", "MLP"],
    "AUC_validación": [best_auc_lgbm, best_auc_xgb, best_auc_mlp],
}

import pandas as pd
df_results = pd.DataFrame(results)
print(df_results.sort_values("AUC_validación", ascending=False))

     Modelo  AUC_validación
2       MLP        0.902740
0  LightGBM        0.900784
1   XGBoost        0.898133


## Comparativa final de modelos

Tras la optimización con Optuna de los tres modelos seleccionados (LightGBM, XGBoost y MLPClassifier), se evaluó su rendimiento en el conjunto de validación para el evento `LiftOff`. La comparación directa de los valores de AUC-ROC obtenidos es la siguiente:

| Modelo     | AUC en validación |
|------------|-------------------|
| **MLP**     | **0.9027**         |
| LightGBM   | 0.9008            |
| XGBoost    | 0.8981            |

El modelo **MLPClassifier** ha demostrado ser el más eficaz en esta tarea tras la optimización de sus hiperparámetros, superando ligeramente a los modelos basados en árboles de decisión.

Por tanto, el siguiente paso consiste en **reentrenar el MLPClassifier óptimo** utilizando **todo el conjunto disponible de entrenamiento** para este evento (`X_train_scaled`, `y_train_win["LiftOff"]`). El objetivo es comparar su rendimiento final frente a la **regresión logística base**, cuyos resultados para cada evento se cargaron previamente desde el archivo `auc_results_feats_logreg.csv` y se almacenaron en el diccionario `auc_dict_logreg`.

Este análisis permitirá determinar si, tras la optimización, el modelo MLP logra superar de forma consistente a la regresión logística —que hasta ahora había ofrecido los mejores resultados en validación— en la predicción de eventos tipo *grasp-and-lift* a partir de señales EEG.

Además, como LightGBM quedó muy cerca también de MLP en resultados, procederemos de la misma manera con este modelo.

In [27]:
import json
import os
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score

# Ruta del archivo de hiperparámetros
save_path = r"C:\Users\luciaft\Documents\TFG\TFG\graspAndLiftDetectionTFGProyect\models"

# Cargar los mejores hiperparámetros desde JSON
with open(os.path.join(save_path, "mlp_liftoff_params.json"), "r") as f:
    best_params = json.load(f)

# Suprimir advertencias de convergencia (opcional)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Función para entrenar y evaluar MLP por evento
def evaluate_mlp_all_events(X_train, y_train, X_valid, y_valid, best_params):
    results = {}
    for event in y_train.columns:
        print(f"Entrenando MLP para evento: {event}")
        clf = MLPClassifier(**best_params)
        clf.fit(X_train, y_train[event])
        y_prob = clf.predict_proba(X_valid)[:, 1]
        auc = roc_auc_score(y_valid[event], y_prob)
        results[event] = auc
        print(f"AUC para {event}: {auc:.4f}")
    return results

# Ejecutar evaluación para todos los eventos
mlp_results_all = evaluate_mlp_all_events(X_train_scaled, y_train_win, X_valid_scaled, y_valid_win, best_params)

Entrenando MLP para evento: HandStart
AUC para HandStart: 0.8495
Entrenando MLP para evento: FirstDigitTouch
AUC para FirstDigitTouch: 0.7828
Entrenando MLP para evento: BothStartLoadPhase


C:\Users\luciaft\OneDrive\Documentos\TFG\grasp_and_lift_proyect\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


AUC para BothStartLoadPhase: 0.9079
Entrenando MLP para evento: LiftOff
AUC para LiftOff: 0.8369
Entrenando MLP para evento: Replace
AUC para Replace: 0.8827
Entrenando MLP para evento: BothReleased
AUC para BothReleased: 0.8436


Como podemos ver, los resultados son notablemente peores. Se nos ocurre que puede ser por el tamaño tan reducido de muestras con el que deicdimos trabajar, por lo que probamos a utilizar Optuna con el tamaño completo de muestras y evitar así el sobreajuste.

In [ ]:
# Diccionarios de salida
X_train_subsampled_dict = {}
y_train_subsampled_dict = {}

# Función corregida para submuestreo
def submuestreo_balanceado(X, y_event, n_samples=200000, random_state=42):
    idx_0 = y_event[y_event == 0].index
    idx_1 = y_event[y_event == 1].index

    n_per_class = n_samples // 2

    sampled_idx_0 = resample(idx_0, replace=False, n_samples=n_per_class, random_state=random_state)
    sampled_idx_1 = resample(idx_1, replace=False, n_samples=n_per_class, random_state=random_state)

    idx_total = np.concatenate([sampled_idx_0, sampled_idx_1])
    np.random.shuffle(idx_total)

    return X.loc[idx_total], y_event.loc[idx_total]

# Loop corregido por evento
for evento in eventos:
    print(f"Procesando evento: {evento}")
    y_event = y_train_win[evento]
    X_event, y_event_bal = submuestreo_balanceado(X_df, y_event)

    X_train_subsampled_dict[evento] = X_event
    y_train_subsampled_dict[evento] = y_event_bal

In [29]:
# === Optuna + MLPClassifier (con todo el conjunto para LiftOff) ===========================

import optuna
from optuna.pruners import MedianPruner
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# Usar todos los datos del evento LiftOff
X = X_train_scaled
y = y_train_win["HandStart"]

def objective(trial):
    # Arquitectura dinámica de capas ocultas
    n_layers = trial.suggest_int("n_layers", 1, 4)
    hidden_layer_sizes = tuple(
        trial.suggest_int(f"n_units_l{i}", 100, 300) for i in range(n_layers)
    )

    # Hiperparámetros
    params = {
        "hidden_layer_sizes": hidden_layer_sizes,
        "activation": trial.suggest_categorical("activation", ["relu", "tanh"]),
        "solver": "adam",
        "alpha": trial.suggest_float("alpha", 1e-6, 1e-2, log=True),
        "learning_rate": trial.suggest_categorical("learning_rate", ["constant", "invscaling", "adaptive"]),
        "learning_rate_init": trial.suggest_float("learning_rate_init", 1e-4, 1e-2, log=True),
        "max_iter": 600,
        "early_stopping": True,
        "random_state": 42,
    }

    # Validación cruzada estratificada
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    aucs = []

    for train_idx, valid_idx in skf.split(X, y):
        X_train_cv, X_valid_cv = X[train_idx], X[valid_idx]
        y_train_cv, y_valid_cv = y.iloc[train_idx], y.iloc[valid_idx]

        model = MLPClassifier(**params)
        model.fit(X_train_cv, y_train_cv)

        y_pred = model.predict_proba(X_valid_cv)[:, 1]
        auc = roc_auc_score(y_valid_cv, y_pred)
        aucs.append(auc)

    return sum(aucs) / len(aucs)

# Estudio Optuna
study = optuna.create_study(direction="maximize", pruner=MedianPruner(n_warmup_steps=5))
study.optimize(objective, n_trials=30, timeout=600)

# Resultados
print(" Mejores hiperparámetros:\n", study.best_params)
print(f" Mejor AUC medio: {study.best_value:.4f}")

[I 2025-05-25 04:25:00,049] A new study created in memory with name: no-name-af57084f-993d-4efe-9463-302480828269
[I 2025-05-25 04:51:52,280] Trial 0 finished with value: 0.9998838763153604 and parameters: {'n_layers': 3, 'n_units_l0': 137, 'n_units_l1': 103, 'n_units_l2': 146, 'activation': 'tanh', 'alpha': 0.008481544019478544, 'learning_rate': 'adaptive', 'learning_rate_init': 0.00033039559110398416}. Best is trial 0 with value: 0.9998838763153604.


 Mejores hiperparámetros:
 {'n_layers': 3, 'n_units_l0': 137, 'n_units_l1': 103, 'n_units_l2': 146, 'activation': 'tanh', 'alpha': 0.008481544019478544, 'learning_rate': 'adaptive', 'learning_rate_init': 0.00033039559110398416}
 Mejor AUC medio: 0.9999


## 🧪Creación del notebook: `08b_optimizacion_avanzada_experimentación.ipynb`


#### PRUEBA 1 

Primeramente probamos implementamos el modelo dando uso de estos últimos hiperparámetros hallados utilizando Optuna con el total de muestras, comenzamos evaluando el modelo **MLP** con los hiperparámetros optimizados previamente para el evento `HandStart`, los cuales habían arrojado un AUC medio de **0.999** durante la validación cruzada interna.

>  Sin embargo, al aplicar este modelo directamente sobre el **conjunto de validación externo**, los resultados fueron notablemente más bajos:

| Evento               | AUC MLP Validación |
|----------------------|--------------------|
| HandStart            | 0.8342             |
| FirstDigitTouch      | 0.8171             |
| BothStartLoadPhase   | 0.8587             |
| LiftOff              | 0.8696             |
| Replace              | 0.8743             |
| BothReleased         | 0.8672             |

>  **Media de AUC-ROC: 0.8702**

Este comportamiento sugiere un claro **sobreajuste** durante el ajuste con Optuna. El modelo logró un rendimiento artificialmente alto sobre los folds internos, pero no fue capaz de generalizar a nuevas muestras del conjunto de validación. 
Esto puede deberse a varios factores:

-  Se utilizó **todo el conjunto de muestras de entrenamiento** (en lugar de un subconjunto balanceado), lo que puede haber inducido al modelo a aprender patrones demasiado específicos.
-  El modelo generó una **arquitectura muy flexible** (3 capas ocultas y muchas unidades), lo cual aumenta el riesgo de *overfitting* si no hay una validación sólida.
-  La **regularización L2 (`alpha`) fue baja**, por lo que no se penalizaron suficientemente los pesos grandes del modelo.
-  Aunque se activó `early_stopping`, este dependía de un *split* interno y no de una validación completamente externa.

Por todo ello, los resultados actuales del MLP, aunque buenos, **no mejoran los de modelos previos más simples**.

---

A continuación, aplicamos el modelo **LightGBM** con hiperparámetros optimizados (también con Optuna, pero entrenado específicamente sobre `LiftOff`) al conjunto de validación completo:

| Evento               | AUC LightGBM Validación |
|----------------------|--------------------------|
| HandStart            | 0.8747                   |
| FirstDigitTouch      | 0.8754                   |
| BothStartLoadPhase   | 0.9017                   |
| LiftOff              | 0.8905                   |
| Replace              | 0.8961                   |
| BothReleased         | 0.8840                   |

>  **Media de AUC-ROC: 0.8871**

A pesar de estar ajustado únicamente para un evento, **LightGBM muestra una generalización sólida** en el resto, logrando:

- Resultados superiores a los del MLP en todos los eventos.
- Un rendimiento cercano al de la **Regresión Logística**, que fue el modelo base con mejor comportamiento (media 0.8905).
- AUC superiores a 0.89 en varios eventos clave como `BothStartLoadPhase` y `Replace`.

---

####  PRUEBA 2: optimizar LightGBM con todas las muestras

Dado su rendimiento competitivo, incluso sin haber sido optimizado para todos los eventos, LightGBM se perfila como un fuerte candidato. 

Optimizarlo más permitirá consolidar un modelo más expresivo, robusto y adaptable a la estructura de los datos EEG temporales y frecuenciales.

De esta forma, dado que LightGBM ya había logrado superar a la Regresión Logística en 3 eventos, vamos a profundizar su optimización para intentar mejorar aún más los resultados:

-  - Utilizar **la mitad del total de muestras disponibles balanceadas** para cada evento (en lugar del subconjunto de 5.000).
-  Aplicaremos **validación cruzada estratificada con 3 folds**, más exigente que la usada previamente.
-  Aumentaremos el **número de `trials` en Optuna (por ejemplo, 100)** y permitiremos más tiempo de ejecución (hasta 40 minutos), ya que la primera exploración fue muy rápida.
-  Incorporaremos un **`MedianPruner`** para descartar combinaciones pobres rápidamente.
-  Evaluaremos los resultados por evento y los compararemos con la Regresión Logística y el MLP.

El objetivo es verificar si **LightGBM puede superar consistentemente a Logistic Regression**, manteniendo al mismo tiempo una buena interpretabilidad y rendimiento computacional razonable.

---

#### PRUEBA 3: 

se probó la optimización de hiperparámetros de **LightGBM** con el algoritmo **Optuna**, pero esta vez utilizando únicamente las **150 características seleccionadas** previamente mediante el algoritmo **Sequential Feature Selector (SFS)**. El objetivo de este experimento era comprobar si **reducir la dimensionalidad** del conjunto de datos podía mejorar el rendimiento del modelo gracias a una menor redundancia y menor ruido.

La hipótesis de partida era que entrenar LightGBM sobre un conjunto reducido de características relevantes podía:

- 🔹 **Reducir el ruido** presente en las 512 características originales, eliminando señales irrelevantes o poco informativas.
- 🔹 **Disminuir la complejidad del modelo**, evitando sobreajuste, especialmente en eventos con menor número de muestras positivas.
- 🔹 **Acelerar la exploración de hiperparámetros**, permitiendo a Optuna concentrarse en combinaciones más relevantes al tener menos variables.
- 🔹 **Aumentar la interpretabilidad**, facilitando el análisis de qué características influyen en cada evento motor.

A pesar de las ventajas teóricas, el modelo entrenado sobre las 150 características **no logró superar el rendimiento de LightGBM entrenado con las 512 características completas**. De hecho, los AUC-ROC por evento fueron en general inferiores. Esto sugiere que:

> Aunque algunas variables originales pudieran parecer poco relevantes individualmente, su combinación con otras aportaba información útil que el modelo sí aprovechaba.

---

Seguimos iterando para mejorar el rendimiento general y detectar si hay algún modelo que supere de forma clara al actual baseline.

Como en el notebook auxiliar de experimentación nos dimos cuenta de que utilizar el total de muestras sin un balanceo nos llevó a un sobreajuste de los hiperparámetros, probamos con submuestreos balanceados al 50% una última vez, tal y como hicimos en el notebook de experimentación, y tras estos últimos resultados dar ya la conclusión final:

In [34]:
import optuna
from optuna.pruners import MedianPruner
from lightgbm import LGBMClassifier, early_stopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample
import numpy as np
import pandas as pd

# --- Evento a optimizar ---
event = "LiftOff"  

# --- Datos completos ---
X_full = X_train_scaled
y_full = y_train_win[event]

# --- Reconstruir DataFrame con los mismos índices que y_train_win ---
X_full_df = pd.DataFrame(X_full, index=y_train_win.index)

# --- Submuestreo balanceado al 50% ---
idx_class_0 = y_full[y_full == 0].index
idx_class_1 = y_full[y_full == 1].index
n_per_class = min(len(idx_class_0), len(idx_class_1)) // 2

idx_sampled_0 = resample(idx_class_0, replace=False, n_samples=n_per_class, random_state=42)
idx_sampled_1 = resample(idx_class_1, replace=False, n_samples=n_per_class, random_state=42)
sampled_idx = np.concatenate([idx_sampled_0, idx_sampled_1])
np.random.shuffle(sampled_idx)

X = X_full_df.loc[sampled_idx]
y = y_full.loc[sampled_idx]

# --- Función objetivo para Optuna ---
def objective(trial):
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 300),
        "max_depth": trial.suggest_int("max_depth", 4, 15),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 100),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 5.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 5.0),
        "n_estimators": 1000,
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1
    }

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    aucs = []

    for train_idx, valid_idx in skf.split(X, y):
        X_train_cv, X_valid_cv = X.iloc[train_idx], X.iloc[valid_idx]
        y_train_cv, y_valid_cv = y.iloc[train_idx], y.iloc[valid_idx]

        model = LGBMClassifier(**params)
        model.fit(
            X_train_cv,
            y_train_cv,
            eval_set=[(X_valid_cv, y_valid_cv)],
            callbacks=[early_stopping(30)],
        )

        y_pred = model.predict_proba(X_valid_cv)[:, 1]
        auc = roc_auc_score(y_valid_cv, y_pred)
        aucs.append(auc)

    return np.mean(aucs)

# --- Estudio Optuna ---
study = optuna.create_study(direction="maximize", pruner=MedianPruner(n_warmup_steps=5))
study.optimize(objective, n_trials=50, timeout=1800)  # 50 trials o 30 minutos

# --- Resultados ---
print(f"\n Mejor AUC para {event}: {study.best_value:.4f}")
print(" Mejores hiperparámetros:")
print(study.best_params)

[I 2025-05-25 07:21:53,759] A new study created in memory with name: no-name-2c21c94a-1b8c-491c-b4da-264bd1ba56f1


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[112]	valid_0's auc: 0.999131
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[130]	valid_0's auc: 0.998865
Training until validation scores don't improve for 30 rounds


[I 2025-05-25 07:22:05,712] Trial 0 finished with value: 0.9990790662241138 and parameters: {'learning_rate': 0.2875422294566247, 'num_leaves': 247, 'max_depth': 13, 'min_child_samples': 34, 'subsample': 0.7242118277397243, 'colsample_bytree': 0.7406133999495319, 'reg_alpha': 0.4533716648844799, 'reg_lambda': 2.4811189809406526}. Best is trial 0 with value: 0.9990790662241138.


Early stopping, best iteration is:
[106]	valid_0's auc: 0.999241
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[117]	valid_0's auc: 0.998933
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[110]	valid_0's auc: 0.998701
Training until validation scores don't improve for 30 rounds


[I 2025-05-25 07:22:16,231] Trial 1 finished with value: 0.9988456644963586 and parameters: {'learning_rate': 0.15997231249903887, 'num_leaves': 296, 'max_depth': 15, 'min_child_samples': 47, 'subsample': 0.802240724049254, 'colsample_bytree': 0.9776174291077502, 'reg_alpha': 4.992781557937632, 'reg_lambda': 0.2250224999078504}. Best is trial 0 with value: 0.9990790662241138.


...

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[774]	valid_0's auc: 0.999315
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[659]	valid_0's auc: 0.998947
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[599]	valid_0's auc: 0.999341


[I 2025-05-25 07:51:06,495] Trial 35 finished with value: 0.9992010330939153 and parameters: {'learning_rate': 0.031645078771366374, 'num_leaves': 222, 'max_depth': 10, 'min_child_samples': 31, 'subsample': 0.8796654183853438, 'colsample_bytree': 0.696378714585456, 'reg_alpha': 0.9292198415887063, 'reg_lambda': 4.171316633196623}. Best is trial 11 with value: 0.9993774925158578.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[827]	valid_0's auc: 0.999322
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[703]	valid_0's auc: 0.999107
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[589]	valid_0's auc: 0.999286


[I 2025-05-25 07:52:35,744] Trial 36 finished with value: 0.9992381073187703 and parameters: {'learning_rate': 0.020805736840702808, 'num_leaves': 256, 'max_depth': 12, 'min_child_samples': 17, 'subsample': 0.7847245168360771, 'colsample_bytree': 0.6266487653032415, 'reg_alpha': 0.5187965635451401, 'reg_lambda': 4.5095229699453965}. Best is trial 11 with value: 0.9993774925158578.



🌟 Mejor AUC para LiftOff: 0.9994
🔧 Mejores hiperparámetros:
{'learning_rate': 0.0382654176447872, 'num_leaves': 180, 'max_depth': 10, 'min_child_samples': 10, 'subsample': 0.8388515908834744, 'colsample_bytree': 0.6114193772926757, 'reg_alpha': 0.0027101804376310767, 'reg_lambda': 4.892457581247462}


In [37]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import numpy as np


# Función para evaluar el modelo por evento
def evaluate_model(model, X_train, y_train, X_valid, y_valid, model_name="Model"):
    results = {}
    for event in y_train.columns:
        print(f"Entrenando {model_name} para evento: {event}")
        model.fit(X_train, y_train[event])
        y_pred_proba = model.predict_proba(X_valid)[:, 1]
        auc = roc_auc_score(y_valid[event], y_pred_proba)
        results[event] = auc
        print(f"{model_name} - {event}: AUC = {auc:.4f}")
    return results

# Crear el modelo con los hiperparámetros
lgbm_final = LGBMClassifier(**study.best_params)

# Evaluar sobre todos los eventos
lgbm_final_results = evaluate_model(
    lgbm_final, X_train_scaled, y_train_win, X_valid_scaled, y_valid_win, model_name="LightGBM"
)

Entrenando LightGBM para evento: HandStart


C:\Users\luciaft\OneDrive\Documentos\TFG\grasp_and_lift_proyect\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


LightGBM - HandStart: AUC = 0.8603
Entrenando LightGBM para evento: FirstDigitTouch


C:\Users\luciaft\OneDrive\Documentos\TFG\grasp_and_lift_proyect\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


LightGBM - FirstDigitTouch: AUC = 0.8935
Entrenando LightGBM para evento: BothStartLoadPhase


...

LightGBM - LiftOff: AUC = 0.8987
Entrenando LightGBM para evento: Replace


C:\Users\luciaft\OneDrive\Documentos\TFG\grasp_and_lift_proyect\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


LightGBM - Replace: AUC = 0.8898
Entrenando LightGBM para evento: BothReleased


C:\Users\luciaft\OneDrive\Documentos\TFG\grasp_and_lift_proyect\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


LightGBM - BothReleased: AUC = 0.8651


## Conclusión Final

### Rendimiento inesperado tras la optimización

A pesar de aplicar técnicas de optimización de hiperparámetros (Optuna) sobre LightGBM utilizando validación cruzada estratificada, los modelos obtenidos **no superaron el rendimiento del modelo entrenado con parámetros por defecto** (`n_estimators=100`, `random_state=42`). Esto sugiere que el modelo base ya se encontraba cerca de su óptimo para este conjunto de datos, bien estructurado y con características eficaces.

Los modelos optimizados tendieron a:
- Sobreajustarse a los folds de validación cruzada
- Utilizar combinaciones innecesariamente complejas
- Perder generalización en el conjunto de validación final

Este hallazgo refuerza la idea de que, en conjuntos de datos con alto volumen y buena ingeniería de características, **los modelos simples pueden ser tan eficaces como los altamente optimizados**. Por ello, se mantendrá la versión base de LightGBM como referencia competitiva frente a Regresión Logística y MLP.
